In [1]:
import pandas as pd
import torch 
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset
import numpy as np
from tqdm import tqdm

In [2]:
path="C:/Quant/project/"
targetpath ="TRAIN.csv"
valpath = "val.csv"
testpath ="test.csv"

In [3]:
df= pd.read_csv(path+valpath)

In [4]:
df

Unnamed: 0  row_id  time_id  investment_id    target       f_0  \
0               0   22_58       22             58  0.600911  1.081210   
1               1   22_59       22             59 -0.148273 -1.107688   
2               2   22_61       22             61 -0.595294  0.776524   
3               3   22_62       22             62 -0.983207 -0.227873   
4               4   22_63       22             63 -0.229776 -0.004463   
...           ...     ...      ...            ...       ...       ...   
24995       24995  33_213       33            213 -1.414616  0.833006   
24996       24996  33_214       33            214 -0.006400 -0.584242   
24997       24997  33_216       33            216 -0.275792  0.087031   
24998       24998  33_217       33            217 -0.934072  0.528163   
24999       24999  33_218       33            218 -0.925161 -0.320688   

            f_1       f_2       f_3       f_4  ...     f_290     f_291  \
0     -1.635753  0.082000 -0.654596  2.116271  ...  1.829991  1.225630   
1     -0.762889  0.759441 -0.661708  0.614619  ...  0.329355  1.225630   
2      0.546407  0.045971 -0.606577 -0.313791  ... -1.076004 -0.815907   
3      0.400930 -1.453166  0.601058  4.339244  ...  0.840153 -0.815907   
4     -0.035502  2.124675 -0.658155 -0.573081  ...  0.808395 -0.815907   
...         ...       ...       ...       ...  ...       ...       ...   
24995 -1.078762 -0.972510  2.457652 -0.235570  ... -0.120991 -0.657001   
24996  0.633364  0.188913 -0.773373 -0.483320  ...  0.446051 -0.657001   
24997  1.204072  0.267470 -0.389980 -0.500376  ... -0.619648 -0.657001   
24998 -1.649470 -1.010666  1.146295 -0.087223  ...  0.394762 -0.657001   
24999  1.917458 -0.796179  1.781502 -0.398205  ... -0.086799 -0.657001   

          f_292     f_293     f_294     f_295     f_296     f_297     f_298  \
0     -0.133157  0.338368 -1.329777  0.458489 -1.171452 -1.116468 -1.403561   
1      0.335490  0.338368  0.752005  0.366660 -1.171452  0.843250  0.520654   
2     -0.349123  0.338368  0.752005 -0.404179  0.084959 -0.250347 -0.819772   
3     -0.667887  0.338368 -1.329777  3.346428 -1.171452 -0.388681  1.211371   
4     -0.425994  0.338368  0.752005 -0.606159 -1.171452  0.717530 -1.274164   
...         ...       ...       ...       ...       ...       ...       ...   
24995 -0.324576  0.173576  0.825183 -0.382438 -0.283733 -0.444719  1.520904   
24996 -0.452254  0.173576  0.825183 -0.401591 -0.283733 -0.329566  0.846726   
24997 -1.341064  0.173576 -1.211851 -0.528420  0.992241 -0.913289 -0.411646   
24998 -0.934546  0.173576  0.825183 -0.255277  0.992241 -0.845721 -0.133167   
24999  0.119822  0.173576  0.825183 -0.417409  0.992241 -1.102599  1.377287   

          f_299  
0      1.496008  
1      0.824550  
2     -0.118027  
3      4.564400  
4     -0.703374  
...         ...  
24995 -0.176580  
24996 -0.484327  
24997 -0.506684  
24998 -0.050718  
24999 -0.373196  

[25000 rows x 305 columns]

In [5]:
#两方面去考虑attention,首先,可以忽视事件数据,将所有Id一样的人聚集在一起,将他们视作一个sequence,将他们的target作为另一个,观察两个sequence的关系
#其次,可以以同一个时域做Attention,将这个时域中的数据作为一个切面sequence,然后跟他们的结果做相关性

In [6]:
def GetDataFromKaggle():
    print("successful get date")

In [24]:
def GetMaxFeature(x):
    id_size = x["investment_id"].nunique()
    time_id_size = x["time_id"].nunique()
    return id_size,time_id_size

In [75]:
class Ubiquant(Dataset):
    def __init__(self,datapath,train_data = True,get_time_data = True):
        super(Ubiquant,self).__init__()
        self.get_time = get_time_data
        if(train_data):
            self.data = pd.read_csv(datapath)
            self.id_size,self.time_size = GetMaxFeature(self.data)
            if(get_time_data):#get an x,y base on time frame
                self.y = self.data.loc[:,"target"]
                self.x = self.data.drop(columns=["Unnamed: 0","row_id","investment_id","target"])
                self.x.loc[:,"time_id"] -=self.x.loc[0,"time_id"]
                self.id = self.x[["time_id"]]
                self.x = self.x.drop(columns=["time_id"])
            else:#get (x,y) base on investment_id
                self.data = self.data.sort_values(by=["investment_id"])
                self.y = self.data.loc[:,"target"]
                self.x = self.data.drop(columns=["Unnamed: 0","row_id","time_id","target"])
                self.id = self.x[["investment_id"]]
                self.x = self.x.drop(columns=["investment_id"])
        
        self.id =torch.LongTensor(self.id.to_numpy())
        self.x = torch.FloatTensor(self.x.to_numpy())
        self.y = torch.FloatTensor(self.y.to_numpy())
        
        print(f"succesfully load data with shape x : {self.x.shape} | shape y :{self.y.shape} | get time data is: {get_time_data}")
        
                
            
    def __getitem__(self,index):
        return self.id[index],self.x[index],self.y[index]
        
    def __len__(self):
        return self.x.shape[0]
    
    def GetSize(self):
        if(get_time_data):
            return self.time_size
        else:
            return self.id_size

            
            

In [76]:
def LoadData(get_time_data = True):
    trainSet =Ubiquant(path+targetpath, get_time_data = get_time_data)
    valSet =Ubiquant(path+valpath, get_time_data = get_time_data)
    testSet =Ubiquant(path+testpath, get_time_data = get_time_data)
    trainLoader = DataLoader(trainSet,batch_size = config["batch_size"],shuffle =True,drop_last=True)#,num_workers=2)
    valLoader = DataLoader(valSet,batch_size = config["batch_size"],shuffle =True,drop_last=True)#,num_workers=2)
    testLoader = DataLoader(testSet,batch_size = config["batch_size"],shuffle =False,drop_last=False)
    return trainLoader,valLoader,testLoader

In [93]:
class SimpleMLP(nn.Module):
    def __init__(self,input_feature = 311):
        super(SimpleMLP,self).__init__()
        
        self.id_embedding = nn.Embedding(10000,11)
        
        self.MLP =nn.Sequential(
            nn.Linear(input_feature,64),
            nn.BatchNorm1d(64),
            nn.SiLU(),
            nn.Dropout(0.4),
            nn.Linear(64,128),
            nn.BatchNorm1d(128),
            nn.SiLU(),
            nn.Dropout(0.4),
            nn.Linear(128,256),
            nn.BatchNorm1d(256),
            nn.SiLU(),
            nn.Dropout(0.4),
            nn.Linear(256,512),
            nn.BatchNorm1d(512),
            nn.SiLU(),
            nn.Dropout(0.4),
            nn.Linear(512,256),
            nn.BatchNorm1d(256),
            nn.SiLU(),
            nn.Dropout(0.4),
            nn.Linear(256,128),
            nn.BatchNorm1d(128),
            nn.SiLU(),
            nn.Dropout(0.4),
            nn.Linear(128,8),
            nn.BatchNorm1d(8),
            nn.SiLU(),
            nn.Dropout(0.4),
            nn.Linear(8,1)
        )
        
        self.criterion = nn.MSELoss()
        
    def forward(self,_id,f_features):
        invest_embedding = self.id_embedding(_id).squeeze(dim=1)
        _input =torch.cat((invest_embedding,f_features),axis =-1)
        output = self.MLP(_input)
        return output

    def cal_Loss(self,y_hat,y):
        return self.criterion(y_hat,y)

In [94]:
def get_corr(y,target):
    y,target = y.reshape(-1),target.reshape(-1)
    ymean,targetmean = torch.mean(y),torch.mean(target)
    
    vy = y-ymean
    vt = target-targetmean
    
    corr = torch.sum(vy*vt)/(torch.sqrt(torch.sum(vy**2))* torch.sqrt(torch.sum(vt**2)))
    return corr

In [95]:
def train(model,trainLoader,valLoader, optimizer, scheduler, config):
    best_loss =1000
    epochs =config["epoch"]
    for epoch in range(epochs):
        batch_bar = tqdm(total = len(trainLoader),dynamic_ncols = True,leave =False,position = 0,desc = "train")
        model.train()
        train_total_loss = 0
        for i,(x1,x2,y) in enumerate(trainLoader):
            optimizer.zero_grad()
            x1,x2,y = x1.cuda(),x2.cuda(),y.cuda()
            y_hat = model(x1,x2)
            y_hat = y_hat.reshape(-1,1)
            y =y.reshape(-1,1)
            loss = model.cal_Loss(y_hat,y)
            train_total_loss+=float(loss.cpu())
            batch_bar.set_postfix(
                loss="{:.04f}".format(float(train_total_loss / (i + 1))),
                lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
            loss.backward()
            optimizer.step()
            scheduler.step()
            batch_bar.update()
        batch_bar.close()
        
        val_total_loss = 0
        val_total_corr = 0
        batch_bar = tqdm(total = len(valLoader),dynamic_ncols = True,leave =False,position = 0,desc = "val")
        model.eval()
        for i,(x1,x2,y) in enumerate(valLoader):
            with torch.no_grad():
                x1,x2,y = x1.cuda(),x2.cuda(),y.cuda()
                y_hat = model(x1,x2)
                y_hat = y_hat.reshape(-1,1)
                y =y.reshape(-1,1)
                loss = model.cal_Loss(y_hat,y)
                

                corr = get_corr(y_hat,y)
                val_total_corr+=corr
               
                val_total_loss+=loss
            batch_bar.set_postfix(
                loss="{:.04f}".format(float(val_total_loss / (i + 1))),
                
                corr="{:.04f}".format(float(val_total_corr / (i + 1))),
             )
            batch_bar.update()
        batch_bar.close()
        val_loss = float(val_total_loss/len(valLoader))
        if(val_loss<best_loss):
            best_loss = val_loss
            torch.save(model.state_dict(),config["store_path"])
            print("successfully save model")
        
        print(f"Epoch {epoch+1}/{epochs}: train loss {float(train_total_loss / len(trainLoader)):0.04f}, Learning Rate {optimizer.param_groups[0]['lr']:0.04f}, val loss{float(val_total_loss/len(valLoader)):0.04f}, corr {float(val_total_corr / (i + 1)):0.04f}")

In [101]:
def Main():
    #set all parameter
    trainLoader,valLoader,testLoader = LoadData(get_time_data = False)
    model = SimpleMLP().cuda()
    model.train()
    optimizer = getattr(torch.optim,config["optimz"])(model.parameters(),lr =config["learning_rate"])
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=(len(trainLoader) * config["epoch"]))
    
    #params
    num_para =0
    for p in model.parameters():
        num_para+=p.numel()
    print(f'Number of params : {num_para}')
    
    train(model,trainLoader,valLoader,optimizer,scheduler,config)

In [104]:
config={"epoch" : 50,"batch_size":12800,"learning_rate":0.1,"optimz":"Adam","store_path":"C:/Quant/project/MLP.pth"}

In [105]:
Main()

succesfully load data with shape x : torch.Size([50000, 300]) | shape y :torch.Size([50000]) | get time data is: False
succesfully load data with shape x : torch.Size([25000, 300]) | shape y :torch.Size([25000]) | get time data is: False
succesfully load data with shape x : torch.Size([25000, 300]) | shape y :torch.Size([25000]) | get time data is: False
Number of params : 470865


successfully save model
Epoch 1/50: train loss 1.0170, Learning Rate 0.0999, val loss763.7325, corr 0.0419


successfully save model
Epoch 2/50: train loss 0.9054, Learning Rate 0.0996, val loss547.5977, corr 0.0379


successfully save model
Epoch 3/50: train loss 0.9019, Learning Rate 0.0991, val loss31.4973, corr -0.0502


successfully save model
Epoch 4/50: train loss 0.8963, Learning Rate 0.0984, val loss4.0075, corr -0.0462


...

Epoch 46/50: train loss 0.8442, Learning Rate 0.0016, val loss0.8931, corr 0.0353


Epoch 47/50: train loss 0.8472, Learning Rate 0.0009, val loss0.8941, corr 0.0580


Epoch 48/50: train loss 0.8414, Learning Rate 0.0004, val loss0.9127, corr 0.0553


Epoch 49/50: train loss 0.8369, Learning Rate 0.0001, val loss0.8965, corr 0.0463


Epoch 50/50: train loss 0.8447, Learning Rate 0.0000, val loss0.8795, corr 0.0586
